In [2]:
import pandas as pd
import numpy as np
#from pyproj import Transformer
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400 # Convert days to seconds

# Step 4: Group points by time window (1-hour threshold)
n = 1
time_threshold = 3600 * n #n hour in seconds

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Define a function to project points onto the Earth sphere surface
def project_to_ecef(x, y, z, radius=6371e3):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm) * radius, (y / norm) * radius, (z / norm) * radius

# Step 6: Create the Earth sphere for reference
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.3, colorscale='Blues', showscale=False, name='Earth'
)

# Step 7: Create frames for animation with projections on the sphere
frames = []

# Initialize data for the first frame
initial_group = groups[0]
initial_data = df.iloc[initial_group]
x_init, y_init, z_init = initial_data['X'], initial_data['Y'], initial_data['Z']

for i, group in enumerate(groups):
    group_data = df.iloc[group]
    x, y, z = group_data['X'], group_data['Y'], group_data['Z']
    proj_x, proj_y, proj_z = project_to_ecef(x, y, z)  # Project points onto Earth

    frame_data = [
    earth,  # Add Earth sphere
    go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=5, color='red', opacity=0.8),
        name=f"Original Group {i}"
    )
    ]
    frames.append(go.Frame(data=frame_data, name=f"Group {i}"))

# Step 8: Layout configuration with Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points and ECEF Projections on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]  
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )],
    #hovermode='closest'
)

# Step 9: Create the figure with Earth sphere and initial points
fig = go.Figure(
    data=[
        earth,  # Static Earth sphere
        go.Scatter3d(
            x=x_init, y=y_init, z=z_init,
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name="Initial Original Points"
        )
    ],
    layout=layout,
    frames=frames
)


# Step 10: Save and open the plot
fig.write_html("satellite_ecef_projection.html", auto_open=True)

#72

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import os
import imageio
from moviepy.editor import ImageSequenceClip

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Group points by time window (1-hour threshold)
n = 1
time_threshold = 3600 * n  # n hours in seconds

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Define a function to project points onto the Earth sphere surface
def project_to_ecef(x, y, z, radius=6371e3):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm) * radius, (y / norm) * radius, (z / norm) * radius

# Step 6: Create the Earth sphere for reference
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.3, colorscale='Blues', showscale=False, name='Earth'
)

# Step 7: Create frames for animation with projections on the sphere
frames = []

# Initialize data for the first frame
for i, group in enumerate(groups):
    group_data = df.iloc[group]
    x, y, z = group_data['X'], group_data['Y'], group_data['Z']
    proj_x, proj_y, proj_z = project_to_ecef(x, y, z)  # Project points onto Earth

    frame_data = [
        earth,  # Add Earth sphere
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Group {i}"
        )
    ]
    frames.append(go.Frame(data=frame_data, name=f"Group {i}"))

# Step 8: Layout configuration with Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points and ECEF Projections on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 9: Create the figure with Earth sphere and initial points
fig = go.Figure(
    data=[
        earth,  # Static Earth sphere
        go.Scatter3d(
            x=df.iloc[groups[0]]['X'],
            y=df.iloc[groups[0]]['Y'],
            z=df.iloc[groups[0]]['Z'],
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name="Initial Original Points"
        )
    ],
    layout=layout,
    frames=frames
)

# Step 10: Save frames for GIF/video
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

for i, frame in enumerate(frames):
    fig.data = frame.data  # Set frame data
    fig.write_image(f"{output_dir}/frame_{i:03d}.png")  # Save each frame as PNG

# Step 11: Create GIF
images = []
for i in range(len(frames)):
    filename = f"{output_dir}/frame_{i:03d}.png"
    images.append(imageio.imread(filename))

gif_path = "satellite_animation.gif"
imageio.mimsave(gif_path, images, duration=0.5)  # Adjust duration as needed
print(f"GIF saved to {gif_path}")

# Step 12: Create Video
fps = 10  # Adjust frame rate as needed
video_path = "satellite_animation.mp4"
clip = ImageSequenceClip(output_dir, fps=fps)
clip.write_videofile(video_path, codec="libx264")
print(f"Video saved to {video_path}")

ModuleNotFoundError: No module named 'moviepy'